# TEST-1

## Test Neo4j Connectivity + Graph Schema

In [1]:
import os
os.getcwd()
os.chdir("c:\\Users\\cherr\\Documents\\MOST-LAB\\AI-Reverse-Engineer")

In [2]:
from storage.graph_store import GraphStore

g = GraphStore(
    uri="neo4j://127.0.0.1:7687",
    user="neo4j",
    password="engineer"
)

## Test IR Objects

In [3]:
from core.ir import FunctionIR, BasicBlock, Instruction

func = FunctionIR(name="main", addr=0x401000)
bb = BasicBlock(addr=0x401020)
instr = Instruction(addr=0x401020, mnemonic="mov", operands=["eax", "1"])


## Test Z3 Agent in Isolation

In [4]:
from agents.z3_agent import Z3Agent

z = Z3Agent()
result = z.check_branch_feasible("eax + 5 == 10")
print(result)

True


## Test Event Bus

In [5]:
from bus.event_bus import EventBus
import time

def handler(event_name, payload):
    print(f"[EVENT] {event_name} -> {payload}")

bus = EventBus()

bus.subscribe("STATIC_ANALYSIS_COMPLETE", handler)
time.sleep(0.2)
bus.publish("STATIC_ANALYSIS_COMPLETE", {"func": "main"})


[EVENT] STATIC_ANALYSIS_COMPLETE -> {'func': 'main'}


[EVENT] STATIC_ANALYSIS_COMPLETE -> {'function_count': 0}


## Test StaticAgent on a PicoCTF Binary 

In [6]:
from agents.static_agent import StaticAgent
from storage.graph_store import GraphStore
from bus.event_bus import EventBus

g = GraphStore(
    uri="bolt://localhost:7687",
    user="neo4j",
    password="engineer",
    database="ai-reverse-db"
)
bus = EventBus()
static = StaticAgent(g, bus)
static.run("tests/binaries/run")

In [7]:
with g.session() as s:
    print(s.run("CALL db.info() YIELD name RETURN name").single())


<Record name='ai-reverse-db'>
